In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import time
import datetime

nltk.download('words')
words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package words to
[nltk_data]     /Users/mosheborochov/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = pd.read_json('../data/labeled_datasets/united-2020-12-05-2020-12-15.json')
print(df['text'])
def stemming(words):
    ps=PorterStemmer()
    return [ps.stem(word) for word in words]

def remove_stop_words(words):
    return [c.lower() for c in words if c not in stop_words]

def lemmatizing(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]


def cleaner(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    return tweet

def final_text(words):
     return ' '.join(words)

df['text'] = df['text'].map(lambda x: cleaner(x))
df['text'] = df['text'].apply(lambda x:word_tokenize(x))
df['text'] = df['text'].apply(lambda x:remove_stop_words(x))
df['text'] = df['text'].apply(lambda x: lemmatizing(x))
df['text'] = df['text'].apply(lambda x: stemming(x))
#df['text'] = df['text'].apply(lambda x:final_text(x))

0        RT @DavidCornDC: Hey @United, a family member ...
1        RT @DavidCornDC: Hey @United, a family member ...
2        RT @DavidCornDC: Worse, @United, your flight a...
3        RT @DavidCornDC: Worse, @United, your flight a...
4        RT @DavidCornDC: Hey @United, a family member ...
                               ...                        
34638    RT @aviacionhr_info: Boeing 737-700 de @united...
34639    @CooperWoods04 @BBOZZY33 @united Just saw this...
34640    RT @aviacionhr_info: Boeing 737-700 de @united...
34641    RT @aviacionhr_info: Boeing 737-700 de @united...
34642    @Obsidian_Knee @TomiLahren @united Lol okay De...
Name: text, Length: 34643, dtype: object


In [4]:
#df[len(df['hashtags'].map(len)) > 0]
df = df.loc[np.array(list(map(len,df.hashtags.values)))>1]

In [5]:
#df = df.explode('hashtags').explode('mentions').explode('text')
#df = df.explode('hashtags').explode('mentions')
#df = df.explode('hashtags')
continuous_index = []
#symbolic_index = ['hashtags', 'user_id', 'mentions', 'text']
#symbolic_index = ['hashtags', 'mentions']
symbolic_index = ['hashtags']

df_continuous = df.loc[:, continuous_index]
df_symbolic = df.loc[:, symbolic_index]
df_label = df.loc[:, ['is_anomaly']]

In [66]:
#df.explode('hashtags')

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly,merlion_anomaly_total_count,merlion_anomaly_top1_hashtag_count,merlion_anomaly_top2_hashtag_count,merlion_anomaly_top3_hashtag_count
106,1336446662491910144,"[first, one, deliveri, 737, max, deliv, 20, mo...",2020-12-08 23:04:48,aviation,"[boeing, united]",NaN,1268941614644371456,0,0,0,12,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
106,1336446662491910144,"[first, one, deliveri, 737, max, deliv, 20, mo...",2020-12-08 23:04:48,avgeek,"[boeing, united]",NaN,1268941614644371456,0,0,0,12,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
174,1336448178636386304,"[the, first, 737max, deliveri, 21, month, n275...",2020-12-08 23:10:50,737max,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
174,1336448178636386304,"[the, first, 737max, deliveri, 21, month, n275...",2020-12-08 23:10:50,n27519,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
295,1336450799661178880,"[boycottunitedairlin, no, point, fli, enforc, ...",2020-12-08 23:21:15,boycottunitedairlines,[united],NaN,222472966,0,0,0,1,NaN,NaN,1.336362e+18,0,0.00000,0.000000,0.506387,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34597,1338987888202027008,"[tuesday, tripl, 1, 400, scale, b777, 200er, n...",2020-12-15 23:22:44,b777,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34597,1338987888202027008,"[tuesday, tripl, 1, 400, scale, b777, 200er, n...",2020-12-15 23:22:44,b777200er,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34597,1338987888202027008,"[tuesday, tripl, 1, 400, scale, b777, 200er, n...",2020-12-15 23:22:44,boeing777,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34623,1338993088379953152,"[want, say, ty, kick, kid, airlin, we, wear, m...",2020-12-15 23:43:24,mask,[united],260907612.0,38445676,0,0,0,1,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0


In [16]:
columns = ['hashtags', 'text']
base_columns = ['id','created_at']
processed_df = pd.DataFrame(columns = columns)
for col in columns:
    tmp_df = df[base_columns+[col]].explode(col)
    for other_col in columns:
        if other_col != col:
            tmp_df[other_col] = 0
    processed_df = pd.concat([processed_df, tmp_df])
processed_df = processed_df.sort_values(by=['id', 'created_at'], ascending = (True, True))


In [17]:
processed_df[['id']] = df[['id']].applymap(np.int64)
processed_df.loc[processed_df['id'] == 1336446662491910144]

,hashtags,text,id,created_at
106,aviation,0,1336446662491910144,2020-12-08 23:04:48
106,avgeek,0,1336446662491910144,2020-12-08 23:04:48
106,0,first,1336446662491910144,2020-12-08 23:04:48
106,0,one,1336446662491910144,2020-12-08 23:04:48
106,0,deliveri,1336446662491910144,2020-12-08 23:04:48
106,0,737,1336446662491910144,2020-12-08 23:04:48
106,0,max,1336446662491910144,2020-12-08 23:04:48
106,0,deliv,1336446662491910144,2020-12-08 23:04:48
106,0,20,1336446662491910144,2020-12-08 23:04:48
106,0,month,1336446662491910144,2020-12-08 23:04:48


In [18]:
processed_df[:10]

,hashtags,text,id,created_at
106,aviation,0,1336446662491910144,2020-12-08 23:04:48
106,avgeek,0,1336446662491910144,2020-12-08 23:04:48
106,0,first,1336446662491910144,2020-12-08 23:04:48
106,0,one,1336446662491910144,2020-12-08 23:04:48
106,0,deliveri,1336446662491910144,2020-12-08 23:04:48
106,0,737,1336446662491910144,2020-12-08 23:04:48
106,0,max,1336446662491910144,2020-12-08 23:04:48
106,0,deliv,1336446662491910144,2020-12-08 23:04:48
106,0,20,1336446662491910144,2020-12-08 23:04:48
106,0,month,1336446662491910144,2020-12-08 23:04:48


In [21]:
df_symbolic.loc[:,'hashtags'].unique()
#df_symbolic.loc[:,'user_id'].unique()
#df_symbolic.loc[:,'mentions'].unique()
#df_symbolic.loc[:,'text'].unique()


hashtags_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'hashtags'].unique()):
    hashtags_dict[entry] = i
    
'''user_id_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'user_id'].unique()):
    user_id_dict[entry] = i'''

'''mentions_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):
    mentions_dict[entry] = i+1
    
text_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):
    text_dict[entry] = i'''

"mentions_dict = {}\nfor i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):\n    mentions_dict[entry] = i+1\n    \ntext_dict = {}\nfor i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):\n    text_dict[entry] = i"

In [22]:
df['created_at'] = pd.to_datetime(df['created_at']).dt.floor('30T')

In [23]:
df

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly,merlion_anomaly_total_count,merlion_anomaly_top1_hashtag_count,merlion_anomaly_top2_hashtag_count,merlion_anomaly_top3_hashtag_count
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:00:00,aviation,"[boeing, united]",NaN,1268941614644371456,0,0,0,12,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:00:00,avgeek,"[boeing, united]",NaN,1268941614644371456,0,0,0,12,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:00:00,737max,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:00:00,n27519,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0,0.00000,0.000000,0.506387,0.0
295,1336450799661178880,#BoycottUnitedAirlines @United No point in fly...,2020-12-08 23:00:00,boycottunitedairlines,[united],NaN,222472966,0,0,0,1,NaN,NaN,1.336362e+18,0,0.00000,0.000000,0.506387,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34597,1338987888202027008,TUESDAY TRIPLE! @GeminiJets 1:400-scale @unite...,2020-12-15 23:00:00,b777,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34597,1338987888202027008,TUESDAY TRIPLE! @GeminiJets 1:400-scale @unite...,2020-12-15 23:00:00,b777200er,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34597,1338987888202027008,TUESDAY TRIPLE! @GeminiJets 1:400-scale @unite...,2020-12-15 23:00:00,boeing777,"[geminijets, united]",NaN,2355209100,0,0,0,24,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0
34623,1338993088379953152,@united just want to say ty for kicking the ki...,2020-12-15 23:30:00,mask,[united],260907612.0,38445676,0,0,0,1,NaN,NaN,NaN,0,-0.54381,-1.000812,0.000000,0.0


In [25]:
df_symbolic.loc[:,'hashtags'] =  df_symbolic.loc[:,'hashtags'].map(hashtags_dict)
#df_symbolic.loc[:,'user_id'] =  df_symbolic.loc[:,'user_id'].map(user_id_dict)
#df_symbolic.loc[:,'mentions'] =  df_symbolic.loc[:,'mentions'].map(mentions_dict)
#df_symbolic.loc[:,'text'] =  df_symbolic.loc[:,'text'].map(text_dict)
#df_symbolic = df_symbolic.fillna(0)
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [26]:
df_continuous.to_csv("CentralParkNYCnumeric.txt", index=False, header=False)
df_symbolic.to_csv("CentralParkNYCcateg.txt", index=False, header=False)
df_label.to_csv("CentralParkNYC_label.txt", index=False, header=False)
def create_unix(x):
    return int(time.mktime((x).timetuple()))
df.loc[:,'created_at'].map(create_unix).to_csv("CentralParkNYCtime.txt", index=False, header=False)
df['id'].to_csv("CentralParkNYC_tweet_id.txt", index=False)

In [43]:
df['id'].to_csv("CentralParkNYC_tweet_id.txt", index=False)

In [27]:
df_symbolic.shape[0]

36171

In [28]:
text_file = open("CentralParkNYCnumeric.txt", "w")
df_symbolic.shape[0]
n = text_file.write('\n'*df_symbolic.shape[0])
text_file.close()

In [29]:
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
df['is_anomaly'].value_counts()

0    21256
1    14915
Name: is_anomaly, dtype: int64

In [30]:
df.shape

(36171, 15)

In [31]:
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [32]:
df_symbolic.iloc[1]

hashtags    0
Name: 1, dtype: int64

In [131]:
score = pd.read_csv('score.txt', header=None)

In [33]:
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [ ]:
df['id'].to_csv("CentralParkNYC_label.txt", index=False, header=False)




In [44]:
df

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
0,1336444683875016704,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:57,NaN,"[DavidCornDC, united]",NaN,779922519285301248,3151,0,0,0,NaN,1.336362e+18,NaN,0
1,1336444690640465920,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:58,NaN,"[DavidCornDC, united]",NaN,161003217,3151,0,0,0,NaN,1.336362e+18,NaN,0
2,1336444721556668416,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:06,NaN,"[DavidCornDC, united]",NaN,161003217,2056,0,0,0,NaN,1.336362e+18,NaN,0
3,1336444729723064320,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:08,NaN,"[DavidCornDC, united]",NaN,794448786,2056,0,0,0,NaN,1.336362e+18,NaN,0
4,1336444732554113024,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:57:08,NaN,"[DavidCornDC, united]",NaN,1165791085626396672,3151,0,0,0,NaN,1.336362e+18,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34638,1338996404480577536,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:56:34,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1191728617623097344,12,0,0,0,NaN,1.338945e+18,NaN,0
34639,1338996604238487552,@CooperWoods04 @BBOZZY33 @united Just saw this...,2020-12-15 23:57:22,NaN,"[CooperWoods04, BBOZZY33, united]",1.168008e+18,2468367223,0,0,1,2,1.337825e+18,NaN,NaN,0
34640,1338996618356535296,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:57:25,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1206188979932016640,12,0,0,0,NaN,1.338945e+18,NaN,0
34641,1338996796312449024,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:58:08,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1333872907517382656,12,0,0,0,NaN,1.338945e+18,NaN,0


In [24]:
df['id'].duplicated().astype(int)

106      0
106      1
174      0
174      1
295      0
        ..
34597    1
34597    1
34597    1
34623    0
34623    1
Name: id, Length: 2290, dtype: int64

In [47]:
df[~df['hashtags'].isnull()]

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
34,1336445155973349376,"In case you’re traveling @united anytime soon,...",2020-12-08 22:58:49,Fixit,[united],NaN,887366707,0,0,0,0,NaN,NaN,1.336362e+18,0
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:04:48,aviation,"[Boeing, united]",NaN,1268941614644371456,2,0,0,12,NaN,NaN,NaN,0
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:04:48,AvGeek,"[Boeing, united]",NaN,1268941614644371456,2,0,0,12,NaN,NaN,NaN,0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:10:50,737MAX,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:10:50,N27519,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34608,1338990361176367104,@seana30914060 @SydneyLWatson @elizfulop @Ryan...,2020-12-15 23:32:33,Plandemic,"[seana30914060, SydneyLWatson, elizfulop, Ryan...",1.154944e+18,1004753288,0,0,1,1,1.338610e+18,NaN,NaN,0
34609,1338990518710231040,.@United are Nazis 🤦🏼‍♂️ Never flying #UnitedA...,2020-12-15 23:33:11,UnitedAirlines,[united],NaN,1315025645664112640,0,0,0,0,NaN,NaN,NaN,0
34622,1338992945383542784,@TomiLahren @united Love that you cry for canc...,2020-12-15 23:42:50,thebiggesthypocrite,"[TomiLahren, united]",4.686470e+08,1248045786841133056,0,0,0,0,1.338239e+18,NaN,NaN,0
34623,1338993088379953152,@united just want to say ty for kicking the ki...,2020-12-15 23:43:24,mask,[united],2.609076e+08,38445676,0,0,0,1,NaN,NaN,NaN,0


In [50]:
df.loc[:,'id']

0        1336444683875016704
1        1336444690640465920
2        1336444721556668416
3        1336444729723064320
4        1336444732554113024
                ...         
34638    1338996404480577536
34639    1338996604238487552
34640    1338996618356535296
34641    1338996796312449024
34642    1338996951048548352
Name: id, Length: 36171, dtype: int64

In [7]:
str(df['hashtags'].iloc[7019])

"['carbonoffset', 'climate', 'renewableenergy', 'sustainable', 'co', 'sequester']"

In [29]:
#df_tmp = df[df['hashtags'].map(lambda d: len(d)) > 0]
#df_tmp[df_tmp['hashtags'].str.contains('united')]
selection = a
df[pd.DataFrame(df['hashtags'].tolist()).isin(selection).any(1).values]

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly,merlion_anomaly_total_count,merlion_anomaly_top1_hashtag_count,merlion_anomaly_top2_hashtag_count,merlion_anomaly_top3_hashtag_count
180,1336448451081596928,[unit],2020-12-08 23:11:55,[united],[united],NaN,929467264881000448,0,0,0,0,NaN,NaN,1.336362e+18,0,0.000000,0.000000,0.506387,0.000000
420,1336453332144189440,"[alarm, read, i, fli, often, reconsid, fli, unit]",2020-12-08 23:31:19,[united],[united],260907612.0,1055607083905830912,0,0,0,0,NaN,NaN,1.336362e+18,0,0.000000,0.000000,0.506387,0.000000
505,1336454917385547776,"[new, destin, airline101, airlin, travel, reno...",2020-12-08 23:37:37,"[airline101, airlines, travel, reno, iah, united]","[renoairport, iah, united]",NaN,381338339,0,0,0,1,NaN,NaN,1.336442e+18,0,0.000000,0.000000,0.506387,0.000000
1707,1336486677209092096,"[whi, i, surpris, unit]",2020-12-09 01:43:49,[united],"[davidcorndc, united]",15220768.0,808139341,0,0,0,0,1.336362e+18,NaN,NaN,0,1.968890,0.000000,0.708957,0.663180
2705,1336516049257320448,"[airlin, news, cut, intern, servic, in, 2021, ...",2020-12-09 03:40:32,"[airline101, airlines, airlinenews, sfo, iad, ...",[united],NaN,381338339,0,0,0,1,NaN,NaN,NaN,0,1.839518,0.000000,0.000000,0.614855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30969,1338483329042100224,"[throw, famili, caus, 2, year, old, wear, mask...",2020-12-14 13:57:48,"[unitedairlines, united]",[united],260907612.0,26162692,0,0,0,0,NaN,NaN,NaN,0,0.000000,-0.647321,-0.526972,0.000000
32185,1338571856635256832,"[31, minut, 5, second, total, phone, charlen, ...",2020-12-14 19:49:34,"[united, changeflights, airtravel, flying]",[united],39697553.0,39697553,0,0,0,1,1.338568e+18,NaN,NaN,0,0.000000,0.000000,0.000000,-0.801993
32623,1338615707320381440,"[3, flight, attend, femal, co, pilot, femal, p...",2020-12-14 22:43:49,[united],[united],NaN,1244391900531920896,0,0,1,2,NaN,NaN,NaN,0,0.000000,-1.337581,1.953390,0.000000
33938,1338882683376635904,"[travel, alert, weather, waiver, airline101, t...",2020-12-15 16:24:41,"[airline101, travelalert, airlines, weather, u...",[united],NaN,381338339,0,0,0,0,NaN,NaN,1.338882e+18,0,-0.609665,-1.011495,-0.868308,0.000000


In [27]:
u = 'united'
a= [u]

In [28]:
a

['united']